In [1]:
from typing import TypedDict, Optional
from langgraph.graph import StateGraph, START, END

from openai import OpenAI
from dotenv import load_dotenv
import os

load_dotenv(override=True)
my_api_key = os.getenv("OPENAI_API_KEY")
# print (f'Key is {my_api_key}')


client = OpenAI(api_key=my_api_key)


# --- Shared State ---
class LibraryState(TypedDict):
    question: Optional[str]
    faq_answer: Optional[str]
    checkout_info: Optional[str]
    final_answer: Optional[str]


def ClassifierAgent(state: LibraryState):
    
    print(f"ClassifierAgent Initial state: {state}")
    
    # state: {'question': 'When does library open?'}
    
    question = state["question"]
    print(f"Classifying question: {question}")

    # Build the LLM messages
    message_to_llm = [
        {"role": "system", "content": '''You are a classifier agent in a library system. 
        Decide if the user is asking about book availability/checkout or about library FAQs. 
        Reply with JSON containing keys: faq_answer and checkout_info.'''},
        {"role": "user", "content": f"Question: {question}"}
    ]
    # Call the OpenAI model
    response = client.chat.completions.create(
        model="gpt-5-nano",
        messages=message_to_llm,
        
    )
    print (response)

    # langchain_chain = prompt | message_to_llm | response_processor


    # Extract the content from the response
    answer = response.choices[0].message.content
    
    print (f"ClassifierAgent function state: {state}")

    # Ideally, parse as JSON — here assuming model returns a dict-like string

    try:
        import json
        parsed = json.loads(answer)
        print(f"Raw response: {answer}")
        print(f"Parsed response: {parsed}")

        return {
            "faq_answer": parsed.get("faq_answer", ""), #parsed["faq_answer"]
            "checkout_info": parsed.get("checkout_info", "")
        }
    except Exception:
        # fallback if LLM gives plain text
        return {"faq_answer": answer, "checkout_info": ""}

def FAQAgent(state: LibraryState):
    
    print(f"FAQAgent Initial state: {state}")
    if not state.get("faq_answer"):
        return {"faq_answer": "Default FAQ: Library rules apply"}
    return {"faq_answer": state["faq_answer"]}


def CheckoutAgent(state: LibraryState):
    print(f"CheckoutAgent Initial state: {state}")
    if not state.get("checkout_info"):
        return {"checkout_info": "Checkout info: Not requested"}
    return { "checkout_info": state["checkout_info"]}


def ResponseAgent(state: LibraryState):
    print(f"CheckouResponseAgent Initial state: {state}")
  
    message_to_llm = [
        {"role": "system", "content": '''You are a response builder for the library application.
          Please combine the FAQ answer and checkout info into a coherent response to the user's question.
        '''},
        {"role": "user", "content": f"FAQ: {state['faq_answer']}\nCheckout Info: {state['checkout_info']}\nQuestion: {state['question']}"}
    ]

    # Call the OpenAI model
    response = client.chat.completions.create(
        model="gpt-5-nano",
        messages=message_to_llm,
     
    )

    print (response)
    # Extract the content from the response
    final_answer = response.choices[0].message.content


    return {"final_answer": final_answer}


# --- Build the Graph ---
builder = StateGraph(LibraryState)
builder.add_node("ClassifierAgent", ClassifierAgent)
builder.add_node("FAQAgent", FAQAgent)
builder.add_node("CheckoutAgent", CheckoutAgent)
builder.add_node("ResponseAgent", ResponseAgent)

builder.add_edge(START, "ClassifierAgent")
builder.add_edge("ClassifierAgent", "FAQAgent")
builder.add_edge("ClassifierAgent", "CheckoutAgent")
builder.add_edge("FAQAgent", "ResponseAgent")
builder.add_edge("CheckoutAgent", "ResponseAgent")
builder.add_edge("ResponseAgent", END)

graph = builder.compile()

In [2]:
result = graph.invoke({"question": "Is The new best seller available? When does library open?"})
print("\n--- Final Answer ---")
print(result["final_answer"])

ClassifierAgent Initial state: {'question': 'Is The new best seller available? When does library open?'}
Classifying question: Is The new best seller available? When does library open?
ChatCompletion(id='chatcmpl-CmPbrNPj0Ncs0lTEbDnRNzWPppNBz', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='{\n  "faq_answer": "This request covers two areas: library hours (FAQ) and book availability (checkout). For hours, please check the library\'s Hours page or ask a librarian for current opening times. For the book, please provide the full title and author so I can check its availability in the catalog; I can also guide you on searching the catalog.",\n  "checkout_info": "To check out or place a hold: 1) search the catalog for the exact title and author, 2) if the item is available, borrow at the desk or via self-checkout with your library card, 3) if not available, place a hold to be notified when it becomes available, 4) you can renew items onli

In [3]:
# # --- Visualize ---
# print(graph.get_graph().draw_ascii())
# graph.get_graph().draw_png("images/agentic_ai_library.png")
# print("Graph saved as agentic_ai_library.png")

In [4]:
# --- Run ---
result = graph.invoke({"question": "When does Cupertino library open? I would like to check out a book as well."})
print("\n--- Final Answer ---")
print(result["final_answer"])

ClassifierAgent Initial state: {'question': 'When does Cupertino library open? I would like to check out a book as well.'}
Classifying question: When does Cupertino library open? I would like to check out a book as well.
ChatCompletion(id='chatcmpl-CmPd2Fx1dFs4zM6MNzsimo6nWAiOR', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='{\n  "faq_answer": "Cupertino Library hours vary by day and season. For the most current opening times, please check the Cupertino Library’s official website or call the branch.",\n  "checkout_info": "To borrow a book, you’ll need a valid library card. You can apply for one online or in person at Cupertino Library. Once you have a card, search the catalog to find a title and see its availability. If the item is on the shelf, you can check it out at a desk or at a self-checkout kiosk using your card. If the item is currently checked out, you can place a hold to be notified when it’s available. You can renew item

In [5]:
result = graph.invoke({"question": "Is The Hobbit available? When does the library open"})
print("\n--- Final Answer ---")
print(result["final_answer"])

ClassifierAgent Initial state: {'question': 'Is The Hobbit available? When does the library open'}
Classifying question: Is The Hobbit available? When does the library open
ChatCompletion(id='chatcmpl-CmPeXjJGULCLedcsFWJnsZ9Yfm18H', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='{\n  "faq_answer": "Opening hours vary by branch. Please specify which library branch you\'re asking about, or say \'any branch\' and I can provide general hours. If you want today’s hours, tell me the branch and I’ll fetch them.",\n  "checkout_info": "To check if \'The Hobbit\' is available, please specify the branch (or \'any\'), the preferred format (physical, ebook, audiobook), and whether you\'d like to place a hold if it’s available."\n}', refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=None))], created=1765654389, model='gpt-5-nano-2025-08-07', object='chat.completion', service_tier='default', system_fingerpr